In [29]:
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plt
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

votes_file = '../assets/datasets/votes.csv'
airport_file = '../assets/datasets/Airport_operations.csv'

# Case #1: Congressional Voting Data

After you've downloaded the data from the repository, go ahead and load it with Pandas

In [30]:
votes = pd.read_csv(votes_file)

In [31]:
votes.shape

(435, 18)

In [32]:
votes.head()

,Unnamed: 0,Class,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,1,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,2,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,3,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,4,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,5,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [33]:
votes = votes.iloc[:, 1:]
votes.head(2)

,Class,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN


In [34]:
Y = [1 if x == 'democrat' else 0 for x in votes.Class.values]
print Y[0:10]

[0, 0, 1, 1, 1, 1, 1, 0, 0, 1]


In [35]:
Xvote = votes.iloc[:, 1:]
Xvote.head(2)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN


In [36]:
Xvote.isnull().sum()

V1      12
V2      48
V3      11
V4      11
V5      15
V6      11
V7      14
V8      15
V9      22
V10      7
V11     21
V12     31
V13     25
V14     17
V15     28
V16    104
dtype: int64

In [37]:
print 'Rows lost:', Xvote.shape[0] - Xvote.dropna().shape[0]
print 'Data points removing rows:', (Xvote.shape[0] - Xvote.dropna().shape[0])*Xvote.shape[1]
print 'Orignal valid data points:', Xvote.shape[0]*Xvote.shape[1] - Xvote.isnull().sum().sum()
print 'INvalid to fill in:', Xvote.isnull().sum().sum()

Rows lost: 203
Data points removing rows: 3248
Orignal valid data points: 6568
INvalid to fill in: 392


In [38]:
def change_yn_to_binary(x):
    if x == 'y':
        return 1
    elif x == 'n':
        return 0
    else:
        return x

# fill in the missing values with random 1 or 0:

# iterate through the columns
for c in Xvote.columns:
    
    Xvote.loc[:, c] = Xvote[c].map(change_yn_to_binary)
    
    # make a mask of rows for this column where the value is null/nan
    mask = Xvote[c].isnull()
    # for those rows at this column, put in random 1s or 0s the size of the number
    # of missing values
    Xvote.loc[mask, c] = np.random.choice(2, size=np.sum(mask))
    
Xvote.isnull().sum()    

//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:545: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.ht

V1     0
V2     0
V3     0
V4     0
V5     0
V6     0
V7     0
V8     0
V9     0
V10    0
V11    0
V12    0
V13    0
V14    0
V15    0
V16    0
dtype: int64

Next, create the covariance matrix from the standardized x-values and decompose these values to find the eigenvalues and eigenvectors

In [39]:
Xn = (Xvote - Xvote.mean()) / Xvote.std()
Xn_corr = Xn.cov()
Xn_corr

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
V1,1.000000,0.034459,0.385787,-0.410596,-0.361700,-0.386424,0.351097,0.380789,0.309473,-0.092997,0.089299,-0.354338,-0.348344,-0.347108,0.182816,0.064338
V2,0.034459,1.000000,-0.062831,0.057807,0.117294,0.146658,-0.196911,-0.085628,-0.172399,-0.140211,0.177925,-0.007338,0.204638,-0.011264,-0.090674,-0.092058
V3,0.385787,-0.062831,1.000000,-0.711485,-0.638523,-0.421310,0.561833,0.674433,0.576725,0.017892,0.211376,-0.607102,-0.492226,-0.568356,0.448372,0.238984
V4,-0.410596,0.057807,-0.711485,1.000000,0.749448,0.469028,-0.577480,-0.680253,-0.624108,0.051374,-0.270819,0.642084,0.576503,0.617912,-0.510065,-0.213288
V5,-0.361700,0.117294,-0.638523,0.749448,1.000000,0.610498,-0.682137,-0.813308,-0.742859,0.015837,-0.131381,0.581911,0.618327,0.658690,-0.542620,-0.220368
V6,-0.386424,0.146658,-0.421310,0.469028,0.610498,1.000000,-0.530016,-0.533405,-0.541858,0.083903,0.020898,0.488406,0.520839,0.562264,-0.448035,-0.140247
V7,0.351097,-0.196911,0.561833,-0.577480,-0.682137,-0.530016,1.000000,0.690502,0.641190,0.023323,0.063257,-0.492941,-0.556160,-0.490109,0.459532,0.299085
V8,0.380789,-0.085628,0.674433,-0.680253,-0.813308,-0.533405,0.690502,1.000000,0.706985,0.023204,0.133206,-0.557980,-0.575587,-0.596427,0.517533,0.283208
V9,0.309473,-0.172399,0.576725,-0.624108,-0.742859,-0.541858,0.641190,0.706985,1.000000,0.029706,0.034042,-0.540345,-0.499049,-0.504767,0.453398,0.214758
V10,-0.092997,-0.140211,0.017892,0.051374,0.015837,0.083903,0.023323,0.023204,0.029706,1.000000,-0.017099,0.051374,0.038903,0.125619,-0.060528,0.078594


In [40]:
eigenvalues, eigenvectors = np.linalg.eig(Xn_corr.values)

Now, let's check the eigenvalues: 

In [41]:
eigenvalues

array([ 7.08062313,  1.41486572,  1.12169695,  0.88286664,  0.85120904,
        0.76037045,  0.61119767,  0.14785371,  0.54403965,  0.48590147,
        0.47440161,  0.35071999,  0.31378582,  0.26083424,  0.44349724,
        0.25613668])

And the eigenvectors: 

In [43]:
print eigenvectors[0:2]
print len(eigenvectors[0])

[[  1.87047967e-01  -1.83367019e-01   1.99625399e-01  -1.20721708e-01
   -6.98181391e-01  -4.08721382e-01  -1.95167728e-01  -6.18708839e-02
    1.90639773e-03   3.20750680e-01   2.01011839e-01   1.37514999e-01
    1.19583260e-01  -6.34582679e-02   2.95161316e-03   3.24249330e-04]
 [ -5.17736378e-02  -6.18597848e-01  -1.00152724e-01  -2.38833404e-01
   -2.15891691e-01   5.93331701e-01  -1.10502683e-01  -8.78057332e-03
   -2.20635414e-01   3.53860127e-02  -1.91439452e-01  -1.45237712e-02
   -9.35286642e-02   4.28194293e-02   1.68598778e-01   1.04142865e-01]]
16


To find the principal components, find the eigenpairs, and sort them from highest to lowest. 

In [44]:
eigen_pairs = [[e_value, e_vector] for e_value, e_vector in zip(eigenvalues, eigenvectors)]
eigen_pairs = sorted(eigen_pairs, reverse=True)

In [45]:
[e[0] for e in eigen_pairs]

[7.0806231252365661,
 1.4148657166667977,
 1.1216969472171952,
 0.88286664256179992,
 0.85120904046995005,
 0.76037044940546183,
 0.61119767179625462,
 0.5440396526425858,
 0.48590146887914804,
 0.47440161282078341,
 0.44349723709364225,
 0.3507199893817205,
 0.31378582270107197,
 0.26083424268999594,
 0.25613667537771739,
 0.14785370505926787]

Now, calculate the explained variance. Recall the methods we learned in lesson 2.2!

In [46]:
total_variance = np.sum(eigenvalues)
explained_variance = [(e[0]/total_variance)*100 for e in eigen_pairs]
explained_variance

[44.253894532728651,
 8.8429107291675102,
 7.010605920107488,
 5.5179165160112644,
 5.3200565029372013,
 4.7523153087841488,
 3.8199854487266016,
 3.4002478290161702,
 3.0368841804946833,
 2.9650100801299044,
 2.7718577318352713,
 2.191999933635759,
 1.961161391881705,
 1.6302140168124792,
 1.6008542211107379,
 0.92408565662042663]

In [55]:
tmp = pd.DataFrame({'vote':Xvote.iloc[0,:].values,
                    'eigenvector':eigenvectors[0],
                    'PC1_pre_sum':Xvote.iloc[0,:].values*eigenvectors[0]})
tmp

,PC1_pre_sum,eigenvector,vote
0,0.000000,0.187048,0.0
1,-0.183367,-0.183367,1.0
2,0.000000,0.199625,0.0
3,-0.120722,-0.120722,1.0
4,-0.698181,-0.698181,1.0
5,-0.408721,-0.408721,1.0
6,-0.000000,-0.195168,0.0
7,-0.000000,-0.061871,0.0
8,0.000000,0.001906,0.0
9,0.320751,0.320751,1.0


In [56]:
tmp.PC1_pre_sum.sum()

-0.8962765792703072

Now, calculate the explained variance and the Cumulative explained variance

Now, conduct a PCA using scikit learn

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

# Case #2: Airport Delays

In [6]:
air = pd.read_csv('../assets/datasets/Airport_operations.csv')

In [8]:
air.head()
air.shape

(841, 15)

First, let's define the x and y variables: Airport is going to be our "x" variable

Then, standardize the x variable for analysis

Next, create the covariance matrix from the standardized x-values and decompose these values to find the eigenvalues and eigenvectors

Then, check your eigenvalues and eigenvectors:

To find the principal componants, find the eigenpairs, and sort them from highest to lowest. 

Next, Calculate the explained variance

Finally, conduct the PCA - use the results about to guide your selection of "n" componants